In [ ]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt
from sklearn.metrics import r2_score
from pyearth import Earth
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
#import data set and creat feature and target space
df = pd.read_csv(r'Volumetric_features.csv')
X=df.drop(columns=['S.No','dataset','Age'])
y=df['Age']

df.head(10)

In [ ]:
df_new =df.drop(columns=['S.No','dataset'])
correlation_matrix = df_new.corr()
correlation_matrix['Age']

cor_target = abs(correlation_matrix['Age']) 

features_to_drop = cor_target[cor_target<0.2] #script to eliminate features with low correlation
to_drop_frame = features_to_drop.to_frame()
row_names = to_drop_frame.index
row_names_list = list(row_names)
row_names_list.append('Age')
row_names_list.append('S.No')
row_names_list.append('dataset')
y = df['Age'].values
x = df.drop(row_names_list, axis=1)

In [ ]:
#data space for model with and without feature selection
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
X_train2,X_test2,y_train2,y_test2 = train_test_split(x,y,test_size=0.25) #model with feature selection

#scaling features
scaler = StandardScaler()
scaler2 = StandardScaler()
scaler.fit(X_train)
scaler2.fit(X_train2)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train2 = scaler2.transform(X_train2)
X_test2 = scaler2.transform(X_test2)

In [ ]:
#Optimize Model Parameters
learn_rate = [0.001,0.01,0.1]
n_est = [10,50,100,300,500,800,1000]

for i in learn_rate:
    print()
    for j in n_est:
        grad_model = GradientBoostingRegressor(learning_rate=i,n_estimators=j)
        grad_model.fit(X_train,y_train)
        print(f'For learning rate {i} and n_estimators {j}, the model score is {grad_model.score(X_test,y_test):.3f}')

In [ ]:
m_score1=[0.012,0.065,0.125,0.317,0.453,0.588,0.648]
m_score2=[0.126,0.454,0.649,0.817,0.838,0.850,0.854]
m_score3=[0.663,0.838,0.855,0.865,0.869,0.870,0.871]
plt.plot(n_est,m_score1)
plt.plot(n_est,m_score2)
plt.plot(n_est,m_score3)
plt.legend(['LR=0.001','LR=0.01','LR=0.1'])
plt.xlabel('Number of estimators used')
plt.ylabel('R^2')
plt.title('Parameter Optimization')

In [ ]:
#Build models based on optimized parameters
grad_model = GradientBoostingRegressor(learning_rate=0.1,n_estimators=800)
grad_model.fit(X_train,y_train)

mse = mean_squared_error(y_test,grad_model.predict(X_test))
r2 = grad_model.score(X_test,y_test)

grad_model2 = GradientBoostingRegressor(learning_rate=0.1,n_estimators=800)
grad_model2.fit(X_train2,y_train2)

mse2 = mean_squared_error(y_test2,grad_model2.predict(X_test2))
r2_2 = grad_model2.score(X_test2,y_test2)

print(f'Model accuracies w/o feature detection: R^2 value is {r2:.3f} and the MSE is {mse:.3f}') #using all predictors
print(f'Model accuracies w/ feature detection: R^2 value is {r2_2:.3f} and the MSE is {mse2:.3f}') #using cov<0.2 drop

In [ ]:
#feature importance for the model
X_train2 = pd.DataFrame(X_train2, columns=x.columns)

importance = grad_model2.feature_importances_ 
indices=np.argsort(importance)[::-1]
#for i,v in enumerate(importance):
    #print('Feature: %0d, Score: %.5f'% (i,v))
plt.bar([x for x in range(len(importance))],importance)
#plt.xticks(range(X_train2.shape[1]),X_train2.columns[indices],rotation=90,fontsize=6)
plt.ylabel('Coefficient Weight')
plt.title('Feature Importance')
plt.show()

print('The feature with the highest weighting in the model is WM-hypointensities')
print('Note this was found by printing feature weights and identifying feature name by printing column name in list')